<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB23_M%C3%A9moires_Short_Term_et_Long_Term.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB23 : construction d'agents avec mémoires "Short_Term" et "Long_Term"

## Objectifs :
Implémentation de 2 types de mémoire pour les agents IA :
- **Short-term memory** → context window (messages récents)
- **Long-term memory** → knowledge base persistante (Vector DB)

## Temps estimé :
- 20–30 minutes

**Livrables :**
- Notebook montrant comment un agent rappelle les conversations récentes et récupère des faits stockés en mémoire long terme.

## Step 1: Setup (5 min)

Installation des prérequis et dépendances nécessaires.

In [ ]:
# Installation des dépendances
!pip install -q openai langchain chromadb python-dotenv
!pip install -q -U langchain-chroma langchain-huggingface langchain-core langchain-openai
!pip install -q datasets
!pip install -q transformers
!pip install -q faiss-cpu
!pip install -q langchain
!pip install -q langchain-community


print("✓ Dépendances installées avec succès")

### Configuration de l'API OpenAI

Dans Google Colab, vous pouvez définir votre clé API de deux façons :
1. **Méthode sécurisée (recommandée)** : Utiliser `google.colab.userdata`
2. **Méthode alternative** : Définir directement en variable d'environnement

In [ ]:
import os
from google.colab import userdata

# Récupérer la clé API depuis les secrets Colab
# Pour ajouter : cliquez sur 🔑 dans le panneau de gauche
try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY'] = openai_api_key
    print("✓ Clé API OpenAI chargée depuis les secrets Colab")
except:
    print("⚠ Secrets Colab non configurés. Veuillez ajouter OPENAI_API_KEY.")
    print("Instructions : Cliquez sur 🔑 dans le panneau gauche > Ajouter un nouveau secret")

## Step 2: Short-Term Memory (Context Window) (5 min)

La mémoire court-terme consiste à maintenir l'historique de conversation dans le prompt.

### Concept clé :
- Sans contexte → le modèle "oublie"
- Avec historique → il peut se rappeler des informations précédemment données

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

def chat_with_memory(messages):
    """
    Fonction pour communiquer avec le modèle en gardant l'historique.

    Args:
        messages: Liste de dictionnaires avec 'role' et 'content'

    Returns:
        str: Réponse du modèle
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7
    )
    return response.choices[0].message.content

print("✓ Fonction chat_with_memory définie")

### Expérience 1 : Conversation avec mémoire

Simulons une conversation où l'agent doit se souvenir du nom de l'utilisateur.

In [ ]:
# Initialisation une conversation
conversation = [
    {"role": "system", "content": "You are a helpful tutor. Be concise and friendly."},
    {"role": "user", "content": "My name is Stephane."}
]

# Premier échange
print("🗣️ User: My name is Stephane.")
reply1 = chat_with_memory(conversation)
print(f"🤖 Assistant: {reply1}")

# Ajout de la réponse à l'historique
conversation.append({"role": "assistant", "content": reply1})

# Deuxième message - test de mémoire court-terme
conversation.append({"role": "user", "content": "What is my name?"})
print("\n🗣️ User: What is my name?")
reply2 = chat_with_memory(conversation)
print(f"🤖 Assistant: {reply2}")

conversation.append({"role": "assistant", "content": reply2})

### Expérience 2 : Sans contexte (oubli)

Demandons le nom du utilisateur **SANS l'historique** pour constater qu'effectivement le modèle oublie.

In [ ]:
# Créer une nouvelle conversation SANS contexte précédent
conversation_without_context = [
    {"role": "system", "content": "You are a helpful tutor."},
    {"role": "user", "content": "What is my name?"}
]

print("🗣️ User: What is my name?")
print("\n(Note: Sans contexte précédent dans la conversation)")
reply_without_context = chat_with_memory(conversation_without_context)
print(f"🤖 Assistant: {reply_without_context}")

print("\n" + "="*60)
print("ANALYSE:")
print(f"✓ AVEC contexte: L'agent se souvient du nom (Stephane)")
print(f"✗ SANS contexte: L'agent ne peut pas savoir le nom")
print("="*60)

## Step 3: Long-Term Memory (Vector Store) (10 min)

Persistance d'informations y compris après  la fin des sessions de conversation.

### Concept clé :
- Vector DB (Chroma) stocke les embeddings des documents et persiste les informations
- Lors d'un échange, on effectue une recherche sémantique : "agent memory" → récupère les documents pertinents
-  → Les informations sont persistées entre les sessions

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
#from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

# Initialiser les embeddings OpenAI
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

print("✓ OpenAIEmbeddings initialisé")

### Créer une base de connaissances

Nous allons stocker plusieurs faits sur les agents IA et LangChain.

In [ ]:
# Les 6 documents de notre base de connaissances
knowledge_base = [
    Document(
        page_content="Agentic AI agents use tools and memory to accomplish complex tasks autonomously.",
        metadata={"source": "agentic_ai_basics", "type": "definition"}
    ),
    Document(
        page_content="LangChain is a framework that helps build autonomous agents with memory, tools, and chains.",
        metadata={"source": "langchain_intro", "type": "framework"}
    ),
    Document(
        page_content="RAG (Retrieval-Augmented Generation) improves accuracy by retrieving relevant knowledge before generating responses.",
        metadata={"source": "rag_concept", "type": "technique"}
    ),
    Document(
        page_content="Short-term memory in agents stores recent conversation context in the prompt window.",
        metadata={"source": "agent_memory", "type": "memory_type"}
    ),
    Document(
        page_content="Long-term memory uses vector databases to persistently store and retrieve semantic knowledge.",
        metadata={"source": "agent_memory", "type": "memory_type"}
    ),
    Document(
        page_content="CrewAI orchestrates multiple AI agents to collaborate on complex workflows.",
        metadata={"source": "crewai_framework", "type": "framework"}
    )
]

# On veille à bien utiliser explicitement le nom de collection 'hanlab_long_term_memory'
db = Chroma.from_documents(
    documents=knowledge_base,
    embedding=embeddings,
    collection_name="hanlab_long_term_memory",  # Nom de collection
    persist_directory="./chroma_db"  # Optionnel: persiste les données sur disque
)

print(f"✓ Base vectorielle créée avec {len(knowledge_base)} documents")
print("✓ Collection: 'hanlab_long_term_memory'")

### Expérience 3 : Rechercher dans la mémoire long-terme

Testons une recherche sémantique.

In [ ]:
def recall_from_long_term_memory(query, k=2):
    """
    Récupère les documents les plus pertinents de la mémoire long-terme.

    Args:
        query: Question ou recherche à effectuer
        k: Nombre de documents à retourner

    Returns:
        list: Documents pertinents
    """
    results = db.similarity_search(query, k=k)
    return results

# Test de recherche
query = "What is agent memory?"
print(f"🔍 Query: {query}\n")

results = recall_from_long_term_memory(query, k=2)

print("Résultats récupérés de la mémoire long-terme:")
for i, doc in enumerate(results, 1):
    print(f"\n  [{i}] {doc.page_content}")
    print(f"      (Source: {doc.metadata['source']})")

### Expérience 4 : Ajouter dynamiquement des nouveaux faits

Les agents peuvent continuellement apprendre et étendre leur base de connaissances.

In [ ]:
# On regarde avant enrichissement les faits récupérables :
query_kafka_avant = "Tell me about data streaming platforms"
print(f"🔍 Query: {query_kafka_avant}")
print("\nRésultats (incluant les nouveaux faits):")
results_kafka = recall_from_long_term_memory(query_kafka_avant, k=2)
for i, doc in enumerate(results_kafka, 1):
    print(f"\n  [{i}] {doc.page_content}")
    print(f"      (Source: {doc.metadata['source']})")

In [ ]:
# Nouveaux faits à ajouter dynamiquement
new_facts = [
    Document(
        page_content="Apache Kafka is a distributed streaming platform for building real-time data pipelines.",
        metadata={"source": "kafka_platform", "type": "technology"}
    ),
    Document(
        page_content="Prompt engineering involves carefully crafting instructions to optimize LLM outputs.",
        metadata={"source": "prompt_engineering", "type": "technique"}
    )
]

# Ajouter les nouveaux documents à la base
db.add_documents(new_facts)

print(f"✓ {len(new_facts)} nouveaux faits ajoutés à la mémoire long-terme")

In [ ]:
# Vérifier que les nouveaux faits sont récupérables
query_kafka = "Tell me about data streaming platforms"
print(f"🔍 Query: {query_kafka}")
print("\nRésultats (incluant les nouveaux faits):")
results_kafka = recall_from_long_term_memory(query_kafka, k=2)
for i, doc in enumerate(results_kafka, 1):
    print(f"\n  [{i}] {doc.page_content}")
    print(f"      (Source: {doc.metadata['source']})")

### Attention aux déppréciations :

| Aspect | RetrievalQA (Ancienne)                    | create_retrieval_chain() (Moderne) |
| ------------ | ----------------------------------------- | ---------------------------------- |
| Statut       |    Deprecated depuis v0.1.0python.langchain​ | ✅ Approche courante actuelle       |
| Architecture | Wrapper hérité                            | LCEL natif (plus flexible)         |
| Contrôle     | Limité                                    | Complet et composable              |
| Maintenance  | Arrêtée                                   | Active et améliorée                |
| Performance  | OK                                        | Optimisée                          |

## Step 4: Association de la mémoire Short-Term et de la mémoire Long-Term (10 min)

Cela consiste à combiner les deux systèmes de mémoire : contexte conversationnel + connaissance persistante.

In [ ]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# Initialiser le LLM et le retriever
llm = ChatOpenAI(model="gpt-4o-mini", temperature=1.0)

retriever = db.as_retriever(search_kwargs={"k": 3})

# Création de la chaîne RAG (Retrieval-Augmented Generation)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

print("✓ Chaîne RetrievalQA créée")
print("✓ Combine short-term (contexte) + long-term (retriever) mémoire")
conversation = [
    {"role":"system","content":"You are a teaching AI agent."},
    {"role":"user","content":"Remember my favorite framework is LangChain."},
    {"role":"assistant","content":"Got it, your favorite framework is LangChain."}
]
print("\nConversation:", conversation)
# User asks later
conversation.append({"role":"user","content":"What's my favorite framework and how do agents use memory?"})
print("_______________________________________")

# La 1ère réponse utilise la mémoire court-terme (contexte) :
short_term_ans = chat_with_memory(conversation)
print("\nShort-term:", short_term_ans)
print("_______________________________________")
# La 2nde réponse est enrichie grace à la mémoire long-terme (recherche dans vector DB) :
# print("\nLong-term:", qa_chain.invoke({"query": "How do agents use memory?"}))
long_term_response = qa_chain.invoke({"query": "How do agents use memory?"})
print("\nLong-term: How do agents use memory ?\n")
print(long_term_response["result"])
print("_______________________________________")
# La 3ème réponse reprend la 1ère question itiale sur le framework préféré mais seulement sur la base de la recherche dans vector DB :
# print("\nLong-term:", qa_chain.invoke({"query": "How do agents use memory?"}))
long_term_response = qa_chain.invoke({"query": "What's my favorite framework?"})
print("\nLong-term: What's my favorite framework ?\n")
print(long_term_response["result"])
print("_______________________________________")

### On rajoute l'information dans la base vectorielle

In [ ]:
user_preference = [
    Document(
        page_content="The user's favorite framework is LangChain because it helps build autonomous agents with memory, tools, and chains.",
        metadata={"source": "user_preferences", "type": "user_preferences"}
    ),
    Document(
        page_content="The best framework is Hadoop.",
        metadata={"source": "prompt_engineering", "type": "Fake"}
    )
]

# Ajouter les nouveaux documents à la base
db.add_documents(user_preference)

print(f"✓ {len(new_facts)} nouveaux faits ajoutés à la mémoire long-terme")


### Et on repose la question :

In [ ]:
# La 4ème réponse reprend la 1ère question initiale sur le framework préféré toujours sur la base de la recherche dans vector DB :
long_term_response = qa_chain.invoke({"query": "What's my favorite framework?"})
print("\nLong-term: What's my favorite framework ?\n")
print(long_term_response["result"])
print("_______________________________________")

# La 5ème réponse interroge sur le meilleur framework :
long_term_response = qa_chain.invoke({"query": "What is the best's Framework?"})
print("\nLong-term: What's my favorite framework ?\n")
print(long_term_response["result"])
print("_______________________________________")

### Expérience 5 : Conversation avec hybridation mémoire

**Scénario :** L'utilisateur indique quel est son framework favori (short-term), puis pose une question impliquant une connaissance long-terme.

In [ ]:
# Initialise la conversation
hybrid_conversation = [
    {
        "role": "system",
        "content": "You are an expert teaching AI agent. You remember user preferences and use knowledge about AI frameworks and techniques."
    },
    {
        "role": "user",
        "content": "My favorite framework is LangChain because it helps build autonomous agents."
    }
]

# Première réponse (short-term memory)
print("🗣️ User: My favorite framework is LangChain because it helps build autonomous agents.")
reply_acknowledge = chat_with_memory(hybrid_conversation)
print(f"🤖 Assistant: {reply_acknowledge}")

hybrid_conversation.append({"role": "assistant", "content": reply_acknowledge})

In [ ]:
# Question qui demande à la fois short-term et long-term memory
user_query = "What's my favorite framework and how do agents use memory?"
hybrid_conversation.append({"role": "user", "content": user_query})

print(f"\n🗣️ User: {user_query}")
print("\n" + "="*60)
print("RÉPONSE 1: Utilisant SHORT-TERM MEMORY UNIQUEMENT")
print("="*60)

# Réponse sans long-term memory (juste contexte)
short_term_reply = chat_with_memory(hybrid_conversation)
print(f"\n🤖 Assistant:\n{short_term_reply}")

### Expérience 6 : Court-terme oublie, Long-terme se souvient

**Scenario :** Après réinitialisation de conversation, short-term memory oublie mais long-term memory persiste.

In [ ]:
print("SCENARIO: Réinitialisation de conversation\n")
print("="*60)

# Conversation 1 : L'agent apprend une info
conv_session1 = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "I love Apache Kafka for real-time data processing."}
]
reply_session1 = chat_with_memory(conv_session1)
print(f"Session 1 - User: I love Apache Kafka for real-time data processing.")
print(f"Session 1 - Assistant: {reply_session1}\n")

# Conversation 2 : NOUVELLE CONVERSATION (conversation réinitialisée)
conv_session2 = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What do I love for data processing?"}
]
reply_session2 = chat_with_memory(conv_session2)
print(f"Session 2 - User: What do I love for data processing?")
print(f"Session 2 - Assistant: {reply_session2}")
print(f"\n❌ SHORT-TERM MEMORY: L'agent a OUBLIÉ (conversation réinitialisée)")
print("="*60)

### Expérience 7 : Persistance avec long-term memory

Même après réinitialisation, la long-term memory persiste.

In [ ]:
user_preference = [
    Document(
        page_content="I love Apache Kafka for real-time data processing..",
        metadata={"source": "user_preferences", "type": "user_preferences"}
    ),
    Document(
        page_content="One another best framework is Trunk Data Platform.",
        metadata={"source": "prompt_engineering", "type": "Tech"}
    )
]

# Ajouter les nouveaux documents à la base
db.add_documents(user_preference)

print(f"✓ {len(new_facts)} nouveaux faits ajoutés à la mémoire long-terme")

In [ ]:
print("SCENARIO: Requête après réinitialisation avec LONG-TERM MEMORY\n")
print("="*60)

# Même question, mais avec RAG
query_persistent = "What technologies are mentioned for real-time processing?"
print(f"Query: {query_persistent}")

# Création de la chaîne RAG (Retrieval-Augmented Generation)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

qa_persistent = qa_chain.invoke({"query": query_persistent})
result_persistent = qa_persistent["result"]

print(f"\n🤖 Assistant (with long-term memory):\n{result_persistent}")
print(f"\n✓ LONG-TERM MEMORY: Les connaissances PERSISTENT même après reset")
print("="*60)

### Expérience 8 : Tester hallucination vs retrieval

Comparer les réponses avec et sans contexte de retrieval.

In [ ]:
# Imports corrects pour LangChain moderne
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Template pour le RAG
template = """You are an AI expert assistant. Use the following context to answer the question.
If you don't know the answer based on the context, say so.

Context: {context}

Question: {question}

Answer:"""

prompt = ChatPromptTemplate.from_template(template)

# Fonction pour formater les documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Créer la chaîne RAG avec LCEL (LangChain Expression Language)
qa_chain_lcel = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Question qui pourrait causer une hallucination
hallucination_query = "What is the relationship between multi-agent systems and real-time data processing?"

print(f"🔍 Query: {hallucination_query}\n")
print("="*60)
print("RÉPONSE 1: Sans retrieval (risque de hallucination)")
print("="*60)

# Réponse directe (sans RAG)
conv_no_rag = [
    {"role": "system", "content": "You are an AI expert."},
    {"role": "user", "content": hallucination_query}
]
reply_no_rag = chat_with_memory(conv_no_rag)
print(f"\n{reply_no_rag}")

print("\n" + "="*60)
print("RÉPONSE 2: Avec retrieval (grounding knowledge)")
print("="*60)

# Utiliser invoke()
reply_rag = qa_chain_lcel.invoke(hallucination_query)
print(f"\n{reply_rag}")

# Pour récupérer aussi les documents sources
docs = retriever.invoke(hallucination_query)
print(f"\nDocuments utilisés comme source:")
for i, doc in enumerate(docs, 1):
    print(f"  [{i}] {doc.page_content[:200]}...")


In [ ]:
# Traçons chaque étape
question = "What is the relationship between multi-agent systems and real-time data processing?"

# Étape 1: retriever + format
step1_context = retriever.invoke(question)
print("STEP 1 - Documents récupérés:\n")
print(f"  Nombre de docs: {len(step1_context)}")

step1_formatted = "\n\n".join(doc.page_content for doc in step1_context)
print(f"  Contexte formaté:\n\n{step1_formatted[:200]}...\n")

# Étape 2: prompt formatting
step2_input = {
    "context": step1_formatted,
    "question": question
}
step2_prompt = prompt.format(**step2_input)
print("STEP 2 - Prompt formaté:\n")
print(f"{step2_prompt}\n")

# Étape 3: LLM generation
step3_llm_output = llm.invoke(step2_prompt)
print("STEP 3 - Réponse du LLM (AIMessage):")
print(f"  Type: {type(step3_llm_output)}")
print(f"  Contenu: {step3_llm_output.content[:200]}...\n")

# Étape 4: Parser
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
step4_final = parser.invoke(step3_llm_output)  # Utiliser invoke() au lieu de parse()
print("STEP 4 - Réponse finale (str):")
print(f"  Type: {type(step4_final)}")
print("  Contenu:")
# Indenter chaque ligne de la réponse
for line in step4_final.split('\n'):
    print(f"    {line}")



### Expérience 9 : Ajouter des faits dynamiquement et tester

Simuler un agent qui apprend continuellement.

In [ ]:
print("SCENARIO: Apprentissage continu (agent learns new facts)\n")
print("="*60)

# Ajouter un nouveau fait important
fact_to_learn =  [
    Document (
        page_content="Function calling enables AI agents to interact with external APIs and tools to perform real-world actions.",
        metadata={"source": "agent_capabilities", "type": "technique"}
)
]

db.add_documents(fact_to_learn)
print("✓ Nouveau fait ajouté: 'Function calling enables AI agents...'")

# Tester la récupération
test_query = "How can agents interact with external systems?"
print(f"\n🔍 Query: {test_query}")

# Invoquer avec la chaîne directement
qa_updated = qa_chain_lcel.invoke(test_query)
print(f"\n🤖 Assistant (with updated knowledge):")
print(qa_updated)

# Afficher aussi les documents sources
docs = retriever.invoke(test_query)
print(f"\n📚 Documents utilisés ({len(docs)} trouvés):")
for i, doc in enumerate(docs, 1):
    print(f"  [{i}] {doc.page_content[:150]}...")

print(f"\n✓ Agent a accès au nouveau fait appris")

## Summary & Key Insights

### Récapitulatif de ce que vous avez appris :

In [ ]:
summary = """\n╔════════════════════════════════════════════════════════════════════╗
║         SHORT-TERM vs LONG-TERM MEMORY IN AGENTIC AI              ║
╚════════════════════════════════════════════════════════════════════╝

📌 SHORT-TERM MEMORY (Context Window)
─────────────────────────────────────
✓ Stocke l'historique récent de conversation
✓ Permet au modèle de référencer les messages précédents
✓ Limité par la taille de la context window (ex: 4K, 8K, 128K tokens)
✗ Se réinitialise à chaque nouvelle conversation
✗ Ne persiste pas entre les sessions

Cas d'usage:
• Conversations interactives
• Contexte immédiat et références
• Dialogue naturel

---

💾 LONG-TERM MEMORY (Vector Database)
──────────────────────────────────────
✓ Stocke les connaissances de façon persistante
✓ Récupération sémantique (recherche par similarité)
✓ Scalable à de grands corpus de connaissances
✓ Persiste entre les sessions
✓ Améliore accuracy et réduit hallucination (RAG)
✗ Nécessite une setup supplémentaire
✗ Coût en appels embeddings

Cas d'usage:
• Knowledge bases persistantes
• Fact retrieval
• Context augmentation (RAG)
• Agents autonomes à long terme

---

🔗 HYBRID APPROACH (Short-term + Long-term)
────────────────────────────────────────────
✓ Combine conversational context + persistent knowledge
✓ Meilleure compréhension du contexte utilisateur
✓ Accès à des faits précis stockés
✓ Réduction des hallucinations
✓ Agents plus intelligents et contextuels

Flux:
1. Contexte court-terme: Lire l'historique conversation
2. Récupération long-terme: Chercher docs pertinents via RAG
3. Fusion: Combiner contexte + docs pour meilleure réponse

---

🚀 NEXT STEPS
─────────────
1. Essayez avec vos propres documents (PDFs, web pages)
2. Tunerez embedding model et retriever (top_k, distance threshold)
3. Implémentez un agent loop complet (ReAct pattern)
4. Explorez multi-agent orchestration (CrewAI)
5. Comparez différentes Vector DBs: Chroma vs Pinecone vs Weaviate
"""
print(summary)

## 🎓 Challenges & Questions

Essayez ces variations pour approfondir votre compréhension :

1. **Challenge 1:** Ajoutez 5 nouveaux faits à la base de connaissances. Testez les requêtes complexes qui les combinent.

2. **Challenge 2:** Créez une conversation multi-tour où l'agent doit:
   - Se rappeler le nom de l'utilisateur (short-term)
   - Récupérer des faits sur les frameworks (long-term)
   - Générer une réponse personnalisée

3. **Challenge 3:** Comparez 2 différentes requêtes:
   - L'une qui demande information dans la base (retrievable)
   - L'une qui demande information hors de la base (hallucination risk)

4. **Challenge 4:** Implémentez un feedback loop:
   - L'agent génère une réponse
   - Vous évaluez la confiance (hallucination ou pas)
   - Ajoutez des documents pour améliorer futures requêtes

5. **Challenge 5:** Explorez `similarity_search_with_score()` pour voir les distances d'embedding

## 📖 Ressources supplémentaires

- [LangChain Documentation](https://python.langchain.com/)
- [Chroma Vector Database](https://www.trychroma.com/)
- [OpenAI Embeddings](https://platform.openai.com/docs/models/embeddings)
- [RAG Pattern](https://python.langchain.com/docs/use_cases/question_answering/)
- [CrewAI for Multi-Agent](https://crewai.io/)

In [ ]:
print("\n🎉 Lab terminé!")
print("✅ Vous avez appris:")
print("   • Comment construire une mémoire court-terme (context window)")
print("   • Comment construire une mémoire long-terme (Vector DB)")
print("   • Comment combiner les deux pour des agents intelligents")
print("   • Comment éviter les hallucinations avec RAG")
print("\n📝 Sauvegardez ce notebook pour références futures!")